In [1]:
import numpy as np
import matplotlib.pyplot as plt

# You may change the mhealth_activity module but your algorithm must support the original version
from mhealth_activity import Recording, Trace, Activity, WatchLocation, Path

# For interactive plots, uncomment the following line
# %matplotlib widget
import os
import pandas as pd
import pickle
from tqdm import tqdm
from scipy.fft import fft, fftfreq
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from scipy.signal import find_peaks
from scipy.signal import peak_prominences
from sklearn import svm
from sklearn.metrics import mean_absolute_error,accuracy_score,precision_score,recall_score,confusion_matrix,classification_report,f1_score
from multiprocessing import Pool

In [2]:
#read data

data = pd.read_pickle('data/pickled_and_sorted_training_data.pkl.zst')
data = data[['temperature','labels']]

In [ ]:
# for x in data['altitude']:
#     print(x.values)

test = [x.values for x in data['altitude']]
print(test)
print(test[0])

In [3]:
# def features_extraction_watchpos(df, prefix): 
    
#     FEATURES = ['MIN','MAX','MEAN','RMS','VAR','STD','POWER','PEAK','P2P','CREST FACTOR','SKEW','KURTOSIS',
#             'MAX_f','SUM_f','MEAN_f','VAR_f','PEAK_f','SKEW_f','KURTOSIS_f']

#     for idx, feature in enumerate(FEATURES):
#         FEATURES[idx] = FEATURES[idx] + '_' + prefix
    
#     print(FEATURES)
#     Min=[];Max=[];Mean=[];Rms=[];Var=[];Std=[];Power=[];Peak=[];Skew=[];Kurtosis=[];P2p=[];CrestFactor=[];
#     FormFactor=[]; PulseIndicator=[];
#     Max_f=[];Sum_f=[];Mean_f=[];Var_f=[];Peak_f=[];Skew_f=[];Kurtosis_f=[]
    
#     X = df.values
#     ## TIME DOMAIN ##
#     #list of lists of lists, ugly as fuck but it works 
#     for recording in X:
#         Min.append(np.min(recording[0]))
#         Max.append(np.max(recording[0]))
#         Mean.append(np.mean(recording[0]))
#         Rms.append(np.sqrt(np.mean(recording[0]**2)))
#         Var.append(np.var(recording[0]))
#         Std.append(np.std(recording[0]))
#         Power.append(np.mean(recording[0]**2))
#         Peak.append(np.max(np.abs(recording[0])))
#         P2p.append(np.ptp(recording[0]))
#         CrestFactor.append(np.max(np.abs(recording[0]))/np.sqrt(np.mean(recording[0]**2)))
#         Skew.append(stats.skew(recording[0]))
#         Kurtosis.append(stats.kurtosis(recording[0]))
#         FormFactor.append(np.sqrt(np.mean(recording[0]**2))/np.mean(recording[0]))
#         PulseIndicator.append(np.max(np.abs(recording[0]))/np.mean(recording[0]))
#         ## FREQ DOMAIN ##
#         ft = fft(recording[0])
#         S = np.abs(ft**2)/len(df)
#         Max_f.append(np.max(S))
#         Sum_f.append(np.sum(S))
#         Mean_f.append(np.mean(S))
#         Var_f.append(np.var(S))
        
#         Peak_f.append(np.max(np.abs(S)))
#         Skew_f.append(stats.skew(recording[0]))
#         Kurtosis_f.append(stats.kurtosis(recording[0]))

    
#     #Create dataframe from features
#     df_features = pd.DataFrame(index = [FEATURES], 
#                                data = [Min,Max,Mean,Rms,Var,Std,Power,Peak,P2p,CrestFactor,Skew,Kurtosis,
#                                        Max_f,Sum_f,Mean_f,Var_f,Peak_f,Skew_f,Kurtosis_f])
#     return df_features

def features_extraction_common(inp): 
    df, prefix = inp
    FEATURES = ['Median', 'Numneg', 'Numpos', 'Numabovmed', 'Mean', 'STD', 'MAD', 'Var', 'Min', 'Max', 'SMA', 'Energy', 'IQR', 'Entropy', 'Npeaks', 'avgprom', 'avgpeakdist', 'Sum_f', 'Max_f', 'NPeak_f', 'Avgprom_f','avgpeakdist_f', 'Mean_f', 'Skew_f', 'Kurtosis_f']

    for idx, feature in enumerate(FEATURES):
        FEATURES[idx] = FEATURES[idx] + '_' + prefix
    
    print(FEATURES)
    Median=[];Numneg=[];Numpos=[];Numabovmed=[];
    Min=[];Max=[];Mean=[];Mad=[];Sma=[];Eng=[];Iqr=[];Entr=[];Std=[];Var=[];Kurt=[];Skew=[];Npeaks=[];Avgprom=[]
    Min_d=[];Max_d=[];Mean_d=[];Mad_d=[];Sma_d=[];Eng_d=[];Iqr_d=[];Entr_d=[];Std_d=[];Var_d=[];
    Max_f=[];NPeak_f=[];Avgprom_f=[];Mean_f=[];Skew_f=[];Kurtosis_f=[];Sum_f=[]; Avgpeakdist=[]; Avgpeakdist_f=[];
    
    X = df.values
    ## TIME DOMAIN ##
    #list of lists of lists, ugly as fuck but it works 
    for recording in X:
        Median.append(np.median(recording[0]))
        Numneg.append(np.sum(np.array(recording[0]) < 0, axis=0))
        Numpos.append(np.sum(np.array(recording[0]) > 0, axis=0))
        Numabovmed.append(np.sum(np.array(recording[0]) > np.median(recording[0]), axis=0))

        Mean.append(np.mean(recording[0]))
        Std.append(np.std(recording[0]))
        #median absolute deviation
        Mad.append(stats.median_abs_deviation(recording[0], scale=1))
        Var.append(np.var(recording[0]))
        Min.append(np.min(recording[0]))
        Max.append(np.max(recording[0]))
        #Signal Magnitude Area
        Sma.append(np.sum(recording[0]))
        #energy measure
        Eng.append(np.sum(recording[0]**2)/len(recording[0]))
        Iqr.append(stats.iqr(recording[0]))
        Entr.append(stats.entropy(recording[0]))

        npeaks, _ = find_peaks(recording[0], distance=5)
        Npeaks.append(len(npeaks))
        prom = peak_prominences(recording[0], npeaks)
        Avgprom.append(np.mean(prom))

        Apeakdist = 0
        for i in range(len(npeaks)-1):
            Apeakdist += abs(npeaks[i] - npeaks[i+1])
        Avgpeakdist.append(Apeakdist/(len(npeaks)-1))

        ## FREQ DOMAIN ##
        ft = np.abs(fft(recording[0]))
        Sum_f.append(np.sum(ft))
        Max_f.append(np.max(ft))

        npeaks, _ = find_peaks(ft, distance=5)
        NPeak_f.append(len(npeaks))
        prom = peak_prominences(ft, npeaks)
        Avgprom_f.append(np.mean(prom))

        Apeakdist = 0
        for i in range(len(npeaks)-1):
            Apeakdist += abs(npeaks[i] - npeaks[i+1])
        Avgpeakdist_f.append(Apeakdist/(len(npeaks)-1))

        Mean_f.append(np.mean(ft))
        Skew_f.append(stats.skew(ft))
        Kurtosis_f.append(stats.kurtosis(ft))

        #derivative
        # f = np.gradient(recording[0])
        # Mean_d.append(np.mean(recording[0]))
        # Std_d.append(np.std(recording[0]))
        # #median absolute deviation
        # Mad_d.append(stats.median_abs_deviation(recording[0], scale=1))
        # Var_d.append(np.var(recording[0]))
        # Min_d.append(np.min(recording[0]))
        # Max_d.append(np.max(recording[0]))
        # #Signal Magnitude Area
        # Sma_d.append(np.sum(recording[0]))
        # #energy measure
        # Eng_d.append(np.sum(recording[0]**2)/len(recording[0]))
        # Iqr_d.append(stats.iqr(recording[0]))
        # Entr_d.append(stats.entropy(recording[0]))

    #Create dataframe from features
    df_features = pd.DataFrame(index = [FEATURES], 
                               data = [Median, Numneg, Numpos, Numabovmed, Mean,Std, Mad, Var, Min, Max, Sma, Eng, Iqr, Entr, Npeaks, Avgprom, Avgpeakdist, Sum_f, Max_f, NPeak_f, Avgprom_f, Avgpeakdist_f, Mean_f, Skew_f, Kurtosis_f]) 
    df_features = pd.DataFrame.transpose(df_features)
    df_features.columns = df_features.columns.map(''.join)
    # get rid of multiindex
    return df_features

# def features_extraction_path(df, prefix): 



In [4]:
#load pickled training 3d norm accelerometer data
file = open('data/accel_mag_train.pkl', 'rb')
pickled = pickle.load(file)
accel_mag_train =  pd.DataFrame(((x,) for x in pickled), columns=['lists'])
print(accel_mag_train.shape)
file.close()

file = open('data/magneto_mag_train.pkl', 'rb')
pickled = pickle.load(file)
magneto_mag_train =  pd.DataFrame(((x,) for x in pickled), columns=['lists'])
print(magneto_mag_train.shape)
file.close()

file = open('data/gyro_mag_train.pkl', 'rb')
pickled = pickle.load(file)
gyro_mag_train =  pd.DataFrame(((x,) for x in pickled), columns=['lists'])
print(gyro_mag_train.shape)
file.close()

pos_labels  = []
path_labels = []
for label in data["labels"]:
    path_labels.extend([label["path_idx"]])
    pos_labels.extend([label["watch_loc"]])


temp_train =  pd.DataFrame(((x.values,) for x in data['temperature']), columns=['temp'])

(396, 1)
(396, 1)
(396, 1)


In [29]:
print(len(gyro_mag_train.loc[20][0]))

34928


In [5]:
#extract features from accelerometer norm

# features_acc = (features_extraction_common((accel_mag_train, "acc")))
# features_mag = (features_extraction_common((magneto_mag_train, "mag")))
# features_gyro = (features_extraction_common((gyro_mag_train, "gyro")))
# features_temp = (features_extraction_common((temp_train, "temp")))

features_to_calculate = ((accel_mag_train, "acc"), (magneto_mag_train, "mag"), (gyro_mag_train, "gyro"), (temp_train, "temp"))

with Pool(4) as p:
    feat = p.map(features_extraction_common, features_to_calculate)

features_acc = feat[0]
features_mag = feat[1]
features_gyro = feat[2]
features_temp = feat[3]

features_acc 



['Median_mag', 'Numneg_mag', 'Numpos_mag', 'Numabovmed_mag', 'Mean_mag', 'STD_mag', 'MAD_mag', 'Var_mag', 'Min_mag', 'Max_mag', 'SMA_mag', 'Energy_mag', 'IQR_mag', 'Entropy_mag', 'Npeaks_mag', 'avgprom_mag', 'avgpeakdist_mag', 'Sum_f_mag', 'Max_f_mag', 'NPeak_f_mag', 'Avgprom_f_mag', 'avgpeakdist_f_mag', 'Mean_f_mag', 'Skew_f_mag', 'Kurtosis_f_mag']
['Median_acc', 'Numneg_acc', 'Numpos_acc', 'Numabovmed_acc', 'Mean_acc', 'STD_acc', 'MAD_acc', 'Var_acc', 'Min_acc', 'Max_acc', 'SMA_acc', 'Energy_acc', 'IQR_acc', 'Entropy_acc', 'Npeaks_acc', 'avgprom_acc', 'avgpeakdist_acc', 'Sum_f_acc', 'Max_f_acc', 'NPeak_f_acc', 'Avgprom_f_acc', 'avgpeakdist_f_acc', 'Mean_f_acc', 'Skew_f_acc', 'Kurtosis_f_acc']
['Median_temp', 'Numneg_temp', 'Numpos_temp', 'Numabovmed_temp', 'Mean_temp', 'STD_temp', 'MAD_temp', 'Var_temp', 'Min_temp', 'Max_temp', 'SMA_temp', 'Energy_temp', 'IQR_temp', 'Entropy_temp', 'Npeaks_temp', 'avgprom_temp', 'avgpeakdist_temp', 'Sum_f_temp', 'Max_f_temp', 'NPeak_f_temp', 'Avgprom

/usr/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,Median_acc,Numneg_acc,Numpos_acc,Numabovmed_acc,Mean_acc,STD_acc,MAD_acc,Var_acc,Min_acc,Max_acc,...,avgprom_acc,avgpeakdist_acc,Sum_f_acc,Max_f_acc,NPeak_f_acc,Avgprom_f_acc,avgpeakdist_f_acc,Mean_f_acc,Skew_f_acc,Kurtosis_f_acc
0,1.192017,0.0,122400.0,61200.0,1.202418,0.212961,0.142390,0.045352,0.153406,2.597480,...,39035.540220,13.566851,2.801477e+06,147175.953766,17363.0,40809.081941,7.049303,22.887887,335.355568,115656.378038
1,1.098560,0.0,117648.0,58824.0,1.092686,0.357553,0.279282,0.127844,0.046882,2.826338,...,39115.644800,18.476437,3.052339e+06,128552.271322,16907.0,39226.357914,6.958476,25.944674,296.831240,96752.692204
2,1.025102,0.0,117568.0,58784.0,1.093724,0.306151,0.161359,0.093728,0.083755,3.057903,...,42167.769097,14.088588,2.960904e+06,128586.928630,16666.0,39198.090598,7.054305,25.184608,308.210927,101873.350901
3,1.015223,0.0,107744.0,53872.0,1.026057,0.349516,0.285567,0.122162,0.276498,2.088655,...,35270.352876,16.671773,2.752588e+06,110551.514205,15408.0,35924.762129,6.992536,25.547485,281.164633,87328.770930
4,1.035623,0.0,76816.0,38408.0,1.078819,0.297570,0.135679,0.088548,0.080601,3.464102,...,25175.700225,9.088037,3.951201e+06,82870.587453,10808.0,25625.830129,7.107245,51.437219,258.451373,69965.964390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,1.295327,0.0,116208.0,58104.0,1.551828,0.640051,0.320108,0.409665,0.093611,3.464102,...,39840.399805,8.405599,1.086863e+07,180334.844977,16511.0,38773.859305,7.038401,93.527378,280.950466,89419.150579
392,1.148992,0.0,113616.0,56808.0,1.468600,0.697000,0.259207,0.485809,0.025811,3.464102,...,38302.608046,9.036268,9.934273e+06,166856.432829,16229.0,37906.648091,7.000863,87.437268,259.084570,79532.373253
393,1.076240,0.0,64512.0,32256.0,1.294628,0.596891,0.212268,0.356279,0.034486,3.464102,...,20601.128547,8.749729,4.710519e+06,83519.058875,9217.0,21533.005236,6.998698,73.017723,203.022719,47528.124409
394,0.937501,0.0,133584.0,66792.0,0.996864,0.410565,0.255059,0.168563,0.032321,3.464066,...,43747.194059,9.534299,5.863363e+06,133165.093887,19138.0,44544.128301,6.979986,43.892708,295.226341,99997.043432


In [6]:
#acc + mag yield the best results(87%), acc + mag + gyro 85%, mag + gyro 85%, acc + gyro  82.5%, 

features = pd.concat([features_acc, features_mag, features_gyro, features_temp], axis=1)


In [7]:
#split the training data because test data doesnt have labels
#why help the students even slightly?

pos_train_features, pos_test_features, pos_train_labels, pos_test_labels = train_test_split(
features, pos_labels, test_size=0.2, random_state=10)

scaler = StandardScaler()
X_train_pos = scaler.fit_transform(pos_train_features) 

# Instantiate model with 1000 decision trees, use all cores
rf_pos = RandomForestRegressor(n_estimators = 1000, random_state = 42, n_jobs=-1)
# Train the model on training data
rf_pos.fit(X_train_pos, pos_train_labels)



RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)

In [8]:
def evaluate_pos(model, test_features, test_labels):
    
    y_pred = model.predict(test_features)

    #cast to int because indices cant be floats
    watch_pos_pred = np.rint(y_pred)

    num_tests = len(watch_pos_pred)

    watch_pos_mismatches = np.count_nonzero(watch_pos_pred != test_labels)

    print(f"{num_tests} tests, {watch_pos_mismatches} watch pos mismatches {(1-(watch_pos_mismatches/num_tests))*100} accuracy")
    return (1-(watch_pos_mismatches/num_tests))*100

scaler = StandardScaler()
X_val = scaler.fit_transform(pos_test_features) 


evaluate_pos(rf_pos,X_val,pos_test_labels)




80 tests, 11 watch pos mismatches 86.25 accuracy


86.25

In [12]:
def test_sensor_combinations():
    for i in range(1,16):
        feat = []
        print(i)
        print("[", end='')
        if(i&1):
            feat.append(features_acc)
            print("acc, ", end='')
        if(i&(1<<1)):
            feat.append(features_mag)
            print("mag, ", end='')
        if(i&(1<<2)):
            feat.append(features_gyro)
            print("gyro, ", end='')
        if(i&(1<<3)):
            feat.append(features_temp)
            print("temp, ", end='')
        print("]")
        features = pd.concat(feat, axis=1)

        pos_train_features, pos_test_features, pos_train_labels, pos_test_labels = train_test_split(
        features, pos_labels, test_size=0.2, random_state=10)

        scaler = StandardScaler()
        X_train_pos = scaler.fit_transform(pos_train_features) 

        # Instantiate model with 1000 decision trees, use all cores
        rf_pos = RandomForestRegressor(n_estimators = 1000, random_state = 42, n_jobs=-1)
        # Train the model on training data
        rf_pos.fit(X_train_pos, pos_train_labels)

        X_val = scaler.fit_transform(pos_test_features) 
        evaluate_pos(rf_pos,X_val,pos_test_labels)

test_sensor_combinations()

1
[acc, ]
80 tests, 59 watch pos mismatches 26.249999999999996 accuracy
2
[mag, ]
80 tests, 57 watch pos mismatches 28.749999999999996 accuracy
3
[acc, mag, ]
80 tests, 58 watch pos mismatches 27.500000000000004 accuracy
4
[gyro, ]
80 tests, 8 watch pos mismatches 90.0 accuracy
5
[acc, gyro, ]
80 tests, 9 watch pos mismatches 88.75 accuracy
6
[mag, gyro, ]


KeyboardInterrupt: 